In [3]:
using Pkg
Pkg.activate(".")
using SymPy
using LinearAlgebra

  Activating project at `c:\Users\tim\Documents\2025FallExamples\12-03`


In [19]:
Dot(a, b) = sum(a .* b)
struct quaternion
    r
    v
end
function quaternion(a0, a1, a2, a3)
    return quaternion(a0, [a1, a2, a3])
end
function norm(q::quaternion)
    return sqrt(q.r^2 + Dot(q.v, q.v))
end
function inv(q::quaternion)
    return quaternion(q.r / norm(q)^2, -q.v / norm(q)^2)
end
function conj(q::quaternion)
    return quaternion( q.r, -q.v)
end
function Base.:+(a::quaternion, b::quaternion)
    return quaternion(a.r + b.r, a.v + b.v)
end
function Base.:*(a::quaternion, b::quaternion)
    return quaternion(a.r * b.r - Dot(a.v, b.v), a.r * b.v + b.r * a.v + cross(a.v, b.v))
end
function Base.:*(a, b::quaternion)
    return quaternion(a * b.r, a * b.v)
end
function Base.:*(b::quaternion, a)
    return quaternion(a * b.r, a * b.v)
end
function Base.:/(b::quaternion, a)
    return quaternion(b.r / a, b.v / a)
end
function q2vec(q::quaternion)
    return vcat([q.r],q.v)
end

tilde(x) = [0 -x[3] x[2]; x[3] 0 -x[1]; -x[2] x[1] 0]
C(q) = q.v * transpose(q.v) + q.r^2 * I - 2 * q.r * tilde(q.v) + tilde(q.v)^2


C (generic function with 1 method)

In [14]:
β0, β1, β2, β3 = symbols("beta_0 beta_1 beta_2 beta_3", real=true)
a0, a1, a2, a3 = symbols("a_0 a_1 a_2 a_3", real=true)
ω1, ω2, ω3 = symbols("omega_1 omega_2 omega_3", real=true);

In [15]:
ωp =quaternion(0,[ω1,ω2,ω3])
β = quaternion(β0,[β1,β2,β3])
a = quaternion(a0,[a1,a2,a3])

quaternion(a_0, Sym{PyCall.PyObject}[a_1, a_2, a_3])

Demonstration: quaternion product using matrices

In [ ]:
La = [ diff(f,x) for f in a*β |> q2vec, x in β |> q2vec ]

4×4 Matrix{Sym{PyCall.PyObject}}:
 a_0  -a_1  -a_2  -a_3
 a_1   a_0  -a_3   a_2
 a_2   a_3   a_0  -a_1
 a_3  -a_2   a_1   a_0

In [25]:
La*q2vec(conj(a)) |> simplify

4-element Vector{Sym{PyCall.PyObject}}:
 a_0^2 + a_1^2 + a_2^2 + a_3^2
                             0
                             0
                             0

Computing $a^{-1}b$ in matrix notation

In [26]:
Ha = [diff(f, x) for f in conj(a) * β |> q2vec, x in β |> q2vec]

4×4 Matrix{Sym{PyCall.PyObject}}:
  a_0   a_1   a_2   a_3
 -a_1   a_0   a_3  -a_2
 -a_2  -a_3   a_0   a_1
 -a_3   a_2  -a_1   a_0

In [27]:
H = Ha[2:end,:]

3×4 Matrix{Sym{PyCall.PyObject}}:
 -a_1   a_0   a_3  -a_2
 -a_2  -a_3   a_0   a_1
 -a_3   a_2  -a_1   a_0

In [29]:
H*q2vec(a)

3-element Vector{Sym{PyCall.PyObject}}:
 0
 0
 0

We see that this array is always singular

In [35]:
H'*I*H |> det

0